# 如何解析 XML 输出

:::info 前置条件

本指南假设您已熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models)
- [输出解析器](/docs/concepts/output_parsers)
- [提示模板](/docs/concepts/prompt_templates)
- [结构化输出](/docs/how_to/structured_output)
- [链接 runnables](/docs/how_to/sequence/)

:::

不同提供商的 LLM 根据其训练数据的不同，通常在特定方面具有不同的优势。这也意味着有些模型在生成 JSON 以外的格式时可能会 "更好" 且更可靠。

本指南向您展示如何使用 [`XMLOutputParser`](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.xml.XMLOutputParser.html) 来提示模型生成 XML 输出，然后 [解析](/docs/concepts/output_parsers/) 该输出为可用格式。

:::note
请记住，大型语言模型是不完全可靠的抽象！您需要使用具有足够能力生成格式良好的 XML 的 LLM。
:::

在以下示例中，我们使用 Anthropic 的 Claude-2 模型 (https://docs.anthropic.com/claude/docs)，这是一个针对 XML 标签进行了优化的模型。

In [ ]:
%pip install -qU langchain langchain-anthropic

import os
from getpass import getpass

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass()

让我们从一个简单的模型请求开始。

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate

model = ChatAnthropic(model="claude-2.1", max_tokens_to_sample=512, temperature=0.1)

actor_query = "生成汤姆·汉克斯的简要电影作品。"

output = model.invoke(
    f"""{actor_query}
请将电影用 <movie></movie> 标签包围"""
)

print(output.content)

Here is the shortened filmography for Tom Hanks, with movies enclosed in XML tags:

<movie>Splash</movie>
<movie>Big</movie>
<movie>A League of Their Own</movie>
<movie>Sleepless in Seattle</movie>
<movie>Forrest Gump</movie>
<movie>Toy Story</movie>
<movie>Apollo 13</movie>
<movie>Saving Private Ryan</movie>
<movie>Cast Away</movie>
<movie>The Da Vinci Code</movie>


这实际上效果很好！但将该 XML 解析为更容易使用的格式会更好。我们可以使用 `XMLOutputParser` 来为提示添加默认格式指令，并将输出的 XML 解析为字典：

In [ ]:
parser = XMLOutputParser()

# 我们将在下面的提示中添加这些指令
parser.get_format_instructions()

'The output should be formatted as a XML file.\n1. Output should conform to the tags below. \n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema. \n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\nNone\n```'

In [4]:
prompt = PromptTemplate(
    template="""{query}\n{format_instructions}""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

output = chain.invoke({"query": actor_query})
print(output)

{'filmography': [{'movie': [{'title': 'Big'}, {'year': '1988'}]}, {'movie': [{'title': 'Forrest Gump'}, {'year': '1994'}]}, {'movie': [{'title': 'Toy Story'}, {'year': '1995'}]}, {'movie': [{'title': 'Saving Private Ryan'}, {'year': '1998'}]}, {'movie': [{'title': 'Cast Away'}, {'year': '2000'}]}]}


我们还可以添加一些标签来定制输出以满足我们的需求。您可以也应该尝试在提示的其他部分添加自己的格式提示，以增强或替换默认指令：

In [ ]:
parser = XMLOutputParser(tags=["movies", "actor", "film", "name", "genre"])

# 我们将在下面的提示中添加这些指令
parser.get_format_instructions()

'The output should be formatted as a XML file.\n1. Output should conform to the tags below. \n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema. \n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\n[\'movies\', \'actor\', \'film\', \'name\', \'genre\']\n```'

In [5]:
prompt = PromptTemplate(
    template="""{query}\n{format_instructions}""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


chain = prompt | model | parser

output = chain.invoke({"query": actor_query})

print(output)

{'movies': [{'actor': [{'name': 'Tom Hanks'}, {'film': [{'name': 'Forrest Gump'}, {'genre': 'Drama'}]}, {'film': [{'name': 'Cast Away'}, {'genre': 'Adventure'}]}, {'film': [{'name': 'Saving Private Ryan'}, {'genre': 'War'}]}]}]}


这个输出解析器还支持部分块的流式传输。这里是一个示例：

In [7]:
for s in chain.stream({"query": actor_query}):
    print(s)

{'movies': [{'actor': [{'name': 'Tom Hanks'}]}]}
{'movies': [{'actor': [{'film': [{'name': 'Forrest Gump'}]}]}]}
{'movies': [{'actor': [{'film': [{'genre': 'Drama'}]}]}]}
{'movies': [{'actor': [{'film': [{'name': 'Cast Away'}]}]}]}
{'movies': [{'actor': [{'film': [{'genre': 'Adventure'}]}]}]}
{'movies': [{'actor': [{'film': [{'name': 'Saving Private Ryan'}]}]}]}
{'movies': [{'actor': [{'film': [{'genre': 'War'}]}]}]}


## 下一步

您现在已经学会了如何提示模型返回 XML。接下来，查看[获取结构化输出的更广泛指南](/docs/how_to/structured_output)以了解其他相关技术。